In [ ]:
from cropmask.download import download_utils as du

In [ ]:
import os
import yaml
config_path = "/home/ryan/work/azure_configs.yaml"

with open(config_path) as f:
    configs = yaml.safe_load(f)

In [ ]:
import lsru

In [ ]:
st = configs['storage']

In [ ]:
espa = du.Espa()

In [ ]:
order = espa.orders[-3]

In [ ]:
order.download_all_complete_azure(st['container'], st['storage_name'], st['storage_key'])

In [ ]:
import os
import yaml
from azure.storage.blob import BlockBlobService
config_path = "/home/ryan/work/azure_configs.yaml"

with open(config_path) as f:
    configs = yaml.safe_load(f)



# Azure Storage Blob Sample - Demonstrate how to use the Blob Storage service. 
# Blob storage stores unstructured data such as text, binary data, documents or media files. 
# Blobs can be accessed from anywhere in the world via HTTP or HTTPS. 
#
 
# Documentation References: 
#  - What is a Storage Account - http://azure.microsoft.com/en-us/documentation/articles/storage-whatis-account/ 
#  - Getting Started with Blobs - https://azure.microsoft.com/en-us/documentation/articles/storage-python-how-to-use-blob-storage/
#  - Blob Service Concepts - http://msdn.microsoft.com/en-us/library/dd179376.aspx 
#  - Blob Service REST API - http://msdn.microsoft.com/en-us/library/dd135733.aspx 
#  - Blob Service Python API - http://azure.github.io/azure-storage-python/ref/azure.storage.blob.html
#  - Storage Emulator - http://azure.microsoft.com/en-us/documentation/articles/storage-use-emulator/ 
#
 


In [ ]:
def run_sample():
    try:
        
        # Create a container called 'quickstartblobs'.
        container_name =configs['storage']['container']
        block_blob_service.create_container(container_name)

        # Set the permission so the blobs are public.
#         block_blob_service.set_container_acl(container_name, public_access=PublicAccess.Container)

        # Create a file in Documents to test the upload and download.
        local_path=os.path.expanduser("~/work/tmp/LT050320312005011601T1-SC20190411001516")
        local_file_name ="LT05_L1TP_032031_20050116_20160913_01_T1_sr_band1.tif"
        full_path_to_file =os.path.join(local_path, local_file_name)


        print("Temp file = " + full_path_to_file)
        print("\nUploading to Blob storage as blob" + local_file_name)

        # Upload the created file, use local_file_name for the blob name
        block_blob_service.create_blob_from_path(container_name, local_file_name, full_path_to_file)

        # List the blobs in the container
        print("\nList blobs in the container")
        generator = block_blob_service.list_blobs(container_name)
        for blob in generator:
            print("\t Blob name: " + blob.name)

    except Exception as e:
        print(e)

In [ ]:
run_sample()

In [ ]:
        # Clean up resources. This includes the container and the temp files
        block_blob_service.delete_container(container_name)
        os.remove(full_path_to_file)
        os.remove(full_path_to_file2)